In [27]:
import datetime as dt
import numpy as np
import pandas as pd
import requests
import json
from pprint import pprint
import csv
# from api_key import api_key
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [36]:
api_key = "93ba5366cb39b249dcbde95cc4d461e7b05b747d"
export_url = (f"https://api.census.gov/data/timeseries/intltrade/exports")
import_url = (f"https://api.census.gov/data/timeseries/intltrade/imports")
china_code = "5700"
usa_code = "1000"
month = []
year = []
#&ParameterName={parameter_name}
#url = (f"https://apps.bea.gov/api/data?&UserID={api_key}&method={get_data}&datasetname={dataset}&LineCode={line_code}&TableName={table_name}&GeoFIPS={geofip}&Year={year}")

#SAINC51
#response = requests.get(url).json()       
#list_response = requests.get(list_url).json()

In [37]:
total_trade = requests.get(f"{export_url}/hs?get=ALL_VAL_MO&key={api_key}&time=2019-01").json() 

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [29]:
total_trade[1][0]


NameError: name 'total_trade' is not defined

In [33]:
requests.get(f"https://api.census.gov/data/timeseries/intltrade/exports/hs?get=DF,ALL_VAL_MO,ALL_VAL_YR&YEAR=2013&MONTH=12").json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [34]:
china_exports = requests.get(f"{export_url}/hs?get=CTY_CODE,CTY_NAME,ALL_VAL_MO,ALL_VAL_YR&key={api_key}&time=from+2018-01&CTY_CODE={china_code}").json()
pprint(china_exports)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [35]:
requests.get(f"{export_url}/hs?get=CTY_CODE,CTY_NAME,ALL_VAL_MO,ALL_VAL_YR,E_COMMODITY_LDESC&key={api_key}&time=from+2015-01&CTY_CODE={china_code}").json()
print("clear")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [8]:
export_by_commodity = requests.get(f"{export_url}/hs?get=E_COMMODITY,E_COMMODITY_SDESC,ALL_VAL_MO,ALL_VAL_YR&key={api_key}&time=from+2015-01&COMM_LVL=HS4&CTY_CODE={china_code}").json()


In [19]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 54072:
    count += 1
    movalue.append(export_by_commodity[count][2])
    yrvalue.append(export_by_commodity[count][3])
    desc.append(export_by_commodity[count][1])
    period.append(export_by_commodity[count][4])
    code.append(export_by_commodity[count][0])
china_data = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value " : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [20]:

china_data.describe()

,YTD Value,Monthly Value,Period,Commodity,Comm Code
count,54071,54071,54071,54071,54071
unique,47597,45977,49,1207,1208
top,9000,0,2018-12,"KNITTED OR CROCHETED FABRICS, NESOI",2828
freq,34,6111,1164,94,49


In [22]:
engine = create_engine('sqlite:///../data/china_trade.db', echo=False)


In [23]:
#china_data.to_sql('export', con=engine, if_exists='replace') #dont run this again

In [24]:
import_by_commodity = requests.get(f"{import_url}/hs?get=I_COMMODITY,I_COMMODITY_SDESC,GEN_VAL_MO,GEN_VAL_YR&key={api_key}&time=from+2015-01&COMM_LVL=HS4&CTY_CODE={china_code}").json()


In [25]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 54072:
    count += 1
    movalue.append(import_by_commodity[count][2])
    yrvalue.append(import_by_commodity[count][3])
    desc.append(import_by_commodity[count][1])
    period.append(import_by_commodity[count][4])
    code.append(import_by_commodity[count][0])
import_china_data = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value " : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [26]:
#import_china_data.to_sql('export', con=engine, if_exists='replace') 